In [ ]:
# %%capture   # uncomment to hide pip output
!pip install -q --upgrade \
    "transformers==4.45.0" \
    "peft==0.10.0" \
    "bitsandbytes==0.43.2" \
    "accelerate==0.30.1" \
    "sentencepiece>=0.2.0" \
    "datasets>=2.19" \
    --progress-bar off --upgrade-strategy eager

# force Colab to restart the Python process so the new wheels are used
import os, sys, textwrap, json, time, importlib, pkg_resources, subprocess, platform
print("✅  Wheels installed; please *Runtime ▸ Restart* or run the cell below:")


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
pylibcudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 20.0.0 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.7.1 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
langchain-core 0.3.63 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.0 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you hav

In [ ]:
#!pip install -q -U bitsandbytes

In [ ]:
#!pip install fsspec==2025.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [ ]:
#!pip install -q -U transformers datasets peft bitsandbytes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


# Fine Tune PaliGemma with QLoRA for Visual Question Answering

In this tutorial, we will learn about PaliGemma briefly and demonstrate how to fine-tune it using the QLoRA (Quantized Low-Rank Adaptation) technique to achieve improved inference results, helping us create a custom vision-language model tailored to our specific tasks.

PaliGemma is an open Vision-Language Model (VLM) released by Google. It builds on the PaLI (Pathways Language and Image) series and the Gemma family of language models.

While many VLMs are getting larger to handle more tasks, PaliGemma focuses on being smaller and more efficient.


* Compact Size: Only about 3B parameters, significantly smaller than many VLMs.
* Competitive Performance: Can be fine-tuned to match the capabilities of larger models.
* Adaptability: Efficient and easy to customize for various tasks.

## What Is a Vision-Language Model?

A Vision-Language Model (VLM) combines visual and textual understanding, processing images and text simultaneously to grasp their relationship. This enables VLMs to handle tasks like image captioning, visual question answering, and more.

How VLMs Work

Image Processing: A component specifically trained to interpret visual data.
Language Processing: Another component designed to handle textual information.
Joint Understanding: The model learns how visuals and language relate, enabling it to describe images or answer questions about them.
For example, a VLM can look at a picture of a dog and generate a descriptive sentence about it.

As VLMs have evolved, they have become more powerful, with larger models able to handle more complex tasks.

However, recent efforts also focus on creating smaller, more efficient VLMs (e.g., PaliGemma) that can perform just as well without massive computational resources.

## Architecture of PaliGemma
PaliGemma is a simple input+text-in, text-out model that takes an image and text as input and generates text as output. PaliGemma’s architecture is built on three key components:

### Image Encoder

PaliGemma uses the SigLIP (Sigmoid loss for Language-Image Pre-training) checkpoint, a publicly available image encoder.
Specifically, it leverages the “shape optimized” Vision Transformer (ViT So400m).
This model is pre-trained on a large scale using contrastive learning with a sigmoid loss, enabling it to process and understand images effectively.
### Decoder-Only Language Model

PaliGemma integrates the Gemma-2B model, a publicly available pre-trained language model for the language component.
As a decoder-only model, it is designed to generate text based on both image and language inputs, allowing for tasks like image captioning or answering questions about an image.
### Linear Layer Projection

The output tokens from the SigLIP encoder are linearly projected into the same dimension as the Gemma-2B model’s vocabulary tokens.
This linear projection ensures that the image and language features can be seamlessly combined and used for downstream tasks.


## How Is PaliGemma Trained?
Inspired by the Pathways Language and Image (PaLI-3) models, PaliGemma follows the same training stages. The training stages are as follows:

### Stage 0: Unimodal Pretraining

* The unimodal components of PaliGemma are pre-trained individually to leverage established training methods.
* No custom unimodal pre-training is performed; publicly available checkpoints are utilized.
* A SigLIP image encoder, specifically the “shape optimized” ViT-So400m model, is smaller yet effective compared to larger models like ViT-G.
* The model adopts the Gemma-2B decoder-only language model, balancing size and performance.
### Stage 1: Multimodal Pretraining

* This stage combines unimodal models and trains them on various vision-language tasks.
* The focus is on creating a base model that can be fine-tuned well for diverse tasks rather than just aligning modalities.
* The image encoder remains frozen initially to maintain representation quality, although it later learns from tasks that provide valuable signals.
* Training occurs at 224px resolution with 1 billion examples, aiming to cover a wide array of concepts and languages.
### Stage 2: Resolution Increase

* The model from Stage 1 is valuable but limited to 224×224 pixel resolution, which is insufficient for tasks requiring higher detail.
* Two additional checkpoints are trained at resolutions of 448×448 and 896×896 pixels.
* Stage 2 emphasizes increasing resolution while retaining the task diversity from Stage 1, with fewer examples but higher information density.
* The text sequence length is increased to 512 tokens for tasks that demand detailed input.
### Stage 3: Transfer

* Stages 1 and 2 produce checkpoints at different resolutions rich in visual knowledge.
* These models need to be fine-tuned for specific tasks to be user-friendly.
* The model can be adapted for various inputs, including multiple images and bounding boxes.


## Available Model Checkpoints
Google has released three types of PaliGemma models:

1. **Pretrained Checkpoints**: These are base models trained on large datasets that can be further fine-tuned for specific downstream tasks.
1. **Mix Checkpoints**: These base models have been fine-tuned on various tasks, making them easy to use with general-purpose free-text prompts.
1. **Fine-Tuned Checkpoints**: These base models have been fine-tuned for specific tasks, providing targeted performance for particular applications.

## Use Cases of PaliGemma
Keeping vision applications in mind, here are some tasks PaliGemma can be used for:

* Image and Short Video Captioning: Asking the model to generate descriptive captions for pictures or short videos.
* Visual Question Answering: Asking the model to generate relevant answers to specific questions about an image.
* Document Understanding: Asking the model to generate answers to specific questions about the text shown in the image.
* Diagram Understanding: Asking the model to generate answers to specific questions about diagrams shown in the image.
* Object Detection: Asking the model to identify (output bounding box coordinates) and label objects within an image.
* Object Segmentation: Asking the model to segment (output polygon coordinates) that bound the detected objects in an image (shown in Figure 2).
* Science Question Answering: Asking the model to provide specific answers to questions related to scientific content, using visual inputs like graphs or diagrams in addition to text.

## Why PaliGemma?
PaliGemma’s compact architecture makes it highly effective across multiple tasks while keeping the model size manageable.\
Its key components (e.g., an image encoder, a language decoder, and a projection layer) allow it to handle complex tasks without requiring the massive computational power that larger models might need.\
Overall, PaliGemma’s flexibility makes it a powerful tool for any task requiring a deep understanding of images and text.

In [1]:
import transformers

In [ ]:
# Remove any mismatched wheels
!pip uninstall -y -q torch torchvision torchaudio

# Install a consistent trio built for CUDA 12
!pip install -q torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 \
  --index-url https://download.pytorch.org/whl/cu124

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip uninstall -y -q torch torchvision torchaudio
!pip install -q --pre torch torchvision torchaudio \
  --index-url https://download.pytorch.org/whl/nightly/cu124

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 58.8 MB/s eta 0:00:00
ERROR: Cannot install torch and torchvision==0.22.0.dev20250226+cu124 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


We import the necessary libraries.

We import torch to provide support for tensor computation. \
We import requests to send HTTP requests for downloading images from the internet.\
We import Image from PIL to open, manipulate, and process images.\
From the transformers library, we import AutoProcessor and PaliGemmaForConditionalGeneration to work with the PaliGemma model.

In [2]:
import torch
import requests
from PIL import Image
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

We import the notebook_login function from the huggingface_hub library, which authenticates our Hugging Face account in a notebook environment.\
We run notebook_login, which prompts us to enter a Hugging Face API token to log in to access Hugging Face models and datasets.


We define a string variable pretrained_model_id with the identifier for the specific PaliGemma pre-trained model (3B parameters, 224px resolution) from the Hugging Face Model Hub.\
We load the pre-trained processor associated with the specified pretrained_model_id. It is responsible for pre-processing input data (e.g., tokenizing text or resizing images) and post-processing the model’s outputs (e.g., decoding).\
We load the pre-trained PaliGemma model associated with the specified pretrained_model_id. It is responsible for generating text outputs based on the inputs (image + text) provided.

In [4]:
pretrained_model_id = "google/paligemma-3b-pt-224"
processor = AutoProcessor.from_pretrained(pretrained_model_id)
pretrained_model = PaliGemmaForConditionalGeneration.from_pretrained(pretrained_model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

* We define the prompt as a text input.
* We assign the URL of an image to the variable image_file. The image is hosted on the Hugging Face dataset repository.
* We download the image from the URL using the requests library and open it using the PIL library (Image.open()). stream=True ensures that the file is streamed in memory rather than downloaded fully before opening.
* We process the image (raw_image) and the text prompt (prompt) using the AutoProcessor. We convert the image to RGB format and prepare the inputs for the model. return_tensors="pt" returns the processed data as PyTorch tensors to be compatible with the pre-trained model.
* We generate the model’s output based on the processed inputs, with a maximum of 20 new tokens in the response. It tries to answer the prompt using the image and text as inputs.
* We decode the generated output from the model using processor.decode, converting the tokenized output back into human-readable text. skip_special_tokens=True removes any special tokens that the model might have generated (e.g., start/end tokens). [len(prompt):] slices the response to exclude the prompt itself, printing only the generated answer.

In [ ]:
prompt = "What is behind the cat?"
image_file = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/cat.png?download=true"
raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(raw_image.convert("RGB"), prompt, return_tensors="pt")
output = pretrained_model.generate(**inputs, max_new_tokens=20)
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.



antique


# Fine-Tuning PaliGemma on a Custom Dataset
Fine-tuning involves adapting a pre-trained model to a specific task or dataset by training it further on new data. This allows the model to specialize and improve performance on domain-specific tasks.

In our case, we fine-tune the PaliGemma model on a custom visual question answering dataset, vqav2-small, a small subset of the VQA v2.0 dataset hosted by Merve on Hugging Face.

The vqav2_small dataset (shown in Figure 4) consists of 21.4k examples with three features: image, question, and multiple_choice_answer. The model is trained using a combination of the image and question as input, while the multiple_choice_answer serves as the label for fine-tuning.

We import several essential libraries to facilitate our tasks.

First, we import a torch library for tensor computation support. Next, requests allows us to send HTTP requests for downloading images from the internet, while PIL is utilized to open, manipulate, and process these images.

The load_dataset function from the datasets library enables us to load datasets for training and evaluation.

Additionally, we import get_peft_model and LoraConfig from the peft library; the former helps create a model with parameter-efficient fine-tuning (PEFT) techniques, and the latter configures the Low-Rank Adaptation (LoRA) method for fine-tuning.

To simplify the training and evaluation of models, we bring in the Trainer class and set the training configuration using TrainingArguments from the transformers library.

The PaliGemmaProcessor is essential for preparing inputs for the PaliGemma model and managing preprocessing tasks (e.g., tokenization).

For optimizing memory usage and computational efficiency during the training of large models, we utilize BitsAndBytesConfig to configure options related to quantization.

Finally, the AutoProcessor automatically selects the appropriate processor based on the model, and the PaliGemmaForConditionalGeneration class is used to generate outputs based on the input data.

In [5]:
import torch
import requests
from PIL import Image
from datasets import load_dataset
from peft import get_peft_model, LoraConfig
from transformers import Trainer
from transformers import TrainingArguments
from transformers import PaliGemmaProcessor
from transformers import BitsAndBytesConfig
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

In [ ]:

# 1️⃣  Remove any ancient copy that Colab pre-installs
!pip uninstall -y -q datasets

# 2️⃣  Install a current build (≥ 2.19 has the LocalFileSystem patch)
!pip install -q "datasets>=2.20" "pyarrow>=15.0" --upgrade-strategy eager


In [ ]:
from datasets import config as _cfg, utils as _u
_cfg.HF_DATASETS_OFFLINE = False          # force re-download

In [ ]:
!rm -rf ~/.cache/huggingface/datasets

We use the load_dataset function from the datasets library to load the dataset merve/vqav2-small, hosted by Merve on Hugging Face.

In [6]:
ds = load_dataset('caglarmert/full_riscm')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/44521 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [ ]:
ds_dataframe = ds['train'].to_pandas()

NameError: name 'ds' is not defined

## Split the Dataset into Train-Test
We use the train_test_split method to split the dataset into training and testing sets. We specify a test_size of 0.2, meaning that 20% of the data will be allocated for testing.

In [ ]:
split_ds = ds["train"].train_test_split(test_size=0.2)
train_ds = split_ds["train"]
test_ds = split_ds["test"]

We initialize the PaliGemmaProcessor using a pre-trained model specified by model_id and define a custom collate_fn function to preprocess a list of examples from the dataset. This function performs several key tasks:

* Formats each example’s question into a text prompt with special tokens.
* Extracts the multiple-choice answers and stores them in a list called labels.
* Converts each image in the examples to RGB format, collecting them in a list named images.
* Next, we utilize the PaliGemmaProcessor to tokenize the text, images, and labels, returning the processed data as PyTorch tensors with padding to the longest sequence.
* For efficiency, we convert the processed tensors to the bfloat16 data type and move them to the specified device, which is the GPU in this case.
* Finally, the function returns the processed tokens, making them ready for input into the model.

In [ ]:
device = "cuda"
model_id = "google/paligemma-3b-pt-224"

In [ ]:
processor = PaliGemmaProcessor.from_pretrained(model_id)
def collate_fn(examples):
      texts = ["<image> <bos> describe this image.}" for example in examples]
      labels= [example['caption_1'] for example in examples]
      images = [example["image"].convert("RGB") for example in examples]
      tokens = processor(text=texts, images=images, suffix=labels,
      return_tensors="pt", padding="longest")
      tokens = tokens.to(torch.bfloat16).to(device)
      return tokens

## Fine-Tune Only the Text Decoder
Since our dataset closely resembles the ones PaliGemma was initially trained on, there’s no need to fine-tune the image encoder or the multimodal projector. We will focus on fine-tuning only the text decoder.

In [ ]:
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16).to(device)
for param in model.vision_tower.parameters():
      param.requires_grad = False
for param in model.multi_modal_projector.parameters():
      param.requires_grad = False

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Using the QLoRA Technique
Now, we set up the configuration to use the Quantized Low-Rank Adapters (QLoRA) technique for model optimization and fine-tuning.

## BitsAndBytesConfig
* load_in_4bit=True: enables 4-bit quantization to reduce memory consumption.
* bnb_4bit_quant_type="nf4": specifies the quantization type, here using "nf4" for more efficient weight representation.
* bnb_4bit_compute_dtype=torch.bfloat16: sets the computation to use bfloat16 to improve computational efficiency during training.

In [ ]:
bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
)
lora_config = LoraConfig(
      r=16,
      target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
      task_type="CAUSAL_LM",
)

## LoraConfig
* r=8: sets the rank (number of dimensions) for low-rank adaptation.
* target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"]: specifies which modules to fine-tune within the model’s language tower (e.g., query, key, and value projections).
* task_type="CAUSAL_LM": defines the task type for the fine-tuning, in this case, causal language modeling.

In [ ]:
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850


In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')


## Define the Training Arguments
We define the training arguments for fine-tuning a model using the Hugging Face Trainer API, including configurations for the number of epochs, batch size, learning rate, optimizer, logging, and saving checkpoints.

We set up the training configuration by specifying key parameters such as training for 2 epochs, a batch size of 4 per device, and accumulating gradients over 4 steps to simulate a larger batch size.

To ensure stability during early training, we include 2 warmup_steps and use a learning_rate of 2e-5 with a small weight_decay of 1e-6 for regularization. The adamw_hf optimizer is configured with an adam_beta2 value of 0.999, and metrics are logged every 100 steps using logging_steps.

We also save model checkpoints every 1,000 steps (as specified by save_steps), limiting the number of saved checkpoints to 1 (using save_total_limit). The checkpoints are stored in the finetuned_paligemma_vqav2_small directory, as defined in output_dir.

Additionally, we enable bfloat16 precision (bf16=True) to reduce memory usage and disable data loader memory pinning (dataloader_pin_memory=False), which can help decrease memory overhead in specific environments.

Finally, we set report_to=["tensorboard"] to log the training progress.

In [ ]:
args=TrainingArguments(
      num_train_epochs=2,
      per_device_train_batch_size=2,
      gradient_accumulation_steps=4,
      warmup_steps=2,
      learning_rate=2e-5,
      weight_decay=1e-6,
      adam_beta2=0.999,
      logging_steps=10,
      optim="adamw_8bit",
      save_strategy="steps",
      save_steps=100,
      save_total_limit=1,
      output_dir="/content/drive/MyDrive/finetuned_paligemma_riscm_full_2",
      bf16=True,
      dataloader_pin_memory=False,
      report_to=["tensorboard"],
      remove_unused_columns=False,
)

In [ ]:
#!pip install --upgrade "transformers>=4.42" "sentencepiece>=0.2.0" --upgrade-strategy eager

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.5.1-py3-none-any.whl (199 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.5.1 which is incompatible.


In [ ]:
import numpy as np
import transformers.models.paligemma.processing_paligemma as pgp
pgp.np = np            # monkey-patch fixes this session only


## Training
We initialize a Hugging Face Trainer with the specified model, training dataset, data collation function, and training arguments, then start the fine-tuning process.

We create an instance of the Hugging Face Trainer class with the following parameters

The pre-trained model to be fine-tuned.\
The dataset (train_ds) and data collator (collate_fn) to handle batching and preprocessing.\
Training configurations from args to specify details (e.g., batch size and learning rate).

In [ ]:
import numpy as np

trainer = Trainer(
      model=model,
      train_dataset=train_ds,
      data_collator=collate_fn,
      args=args
)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.915900
20,3.844700
30,3.749400
40,3.364800
50,3.108900
60,3.081500
70,2.805900
80,2.683900
90,2.539800
100,2.661100


TrainOutput(global_step=8904, training_loss=0.9310679763475197, metrics={'train_runtime': 16414.2777, 'train_samples_per_second': 4.34, 'train_steps_per_second': 0.542, 'total_flos': 2.9037077749845274e+17, 'train_loss': 0.9310679763475197, 'epoch': 2.0})

## Push the Fine-Tuned PaliGemma Model to the Hub
We upload our fine-tuned model, which includes the safetensors file and model configurations, to the Hub so that it can be used later for inference. The fine-tuned model can be found here: Hugging Face Model Hub.

In [ ]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/45.3M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

events.out.tfevents.1749335705.ff8bdfcb42ee.485.9:   0%|          | 0.00/194k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/orcan/finetuned_paligemma_riscm_full_2/commit/b36a3e3c0b85245365bba9341dff3fd33c8a2a11', commit_message='End of training', commit_description='', oid='b36a3e3c0b85245365bba9341dff3fd33c8a2a11', pr_url=None, repo_url=RepoUrl('https://huggingface.co/orcan/finetuned_paligemma_riscm_full_2', endpoint='https://huggingface.co', repo_type='model', repo_id='orcan/finetuned_paligemma_riscm_full_2'), pr_revision=None, pr_num=None)

# Inference with Fine-Tuned PaliGemma Model
Now, we can use our uploaded fine-tuned PaliGemma model to run inference on examples from our test split dataset.

We first initialize the processor with pretrained_model_id to handle input preprocessing. Next, we load the model using finetuned_model_id, which points to our fine-tuned version of PaliGemma.

In [ ]:
pretrained_model_id = "google/paligemma-3b-pt-224"
#finetuned_model_id = "orcan/finetuned_paligemma_riscm_full_r_16"
processor = AutoProcessor.from_pretrained(pretrained_model_id)
model = PaliGemmaForConditionalGeneration.from_pretrained(pretrained_model_id) # finetuned_model_id

adapter_config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/45.3M [00:00<?, ?B/s]

Here, we have the same inference code as before. We define the text prompt and download the image using requests, then open it with the PIL library (Image.open()). After that, we preprocess the image and prompt using the processor, generate a response from the model, and finally decode the output to get the answer.

When we ran inference with the pre-trained model, the output generated was antique, which wasn’t entirely accurate. However, after fine-tuning, the model generated a gramophone, which is a more accurate result.

In [ ]:
prompt = "Describe this image."
image_file = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/cat.png?download=true"
raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(raw_image.convert("RGB"), prompt, return_tensors="pt")
output = model.generate(**inputs, max_new_tokens=40)
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.



A cat is sleeping on a table .


In [ ]:
prompt = "Describe this image."
image_file = test_ds[1]["image"]
inputs = processor(image_file, prompt, return_tensors="pt")
output = model.generate(**inputs, max_new_tokens=20)
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

NameError: name 'test_ds' is not defined

In [ ]:
prompt = "Describe this image."
image_file = test_ds[2]["image"]
inputs = processor(image_file, prompt, return_tensors="pt")
output = model.generate(**inputs, max_new_tokens=20)
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

In [ ]:
prompt = "Describe this image."
image_file = test_ds[3]["image"]
inputs = processor(image_file, prompt, return_tensors="pt")
output = model.generate(**inputs, max_new_tokens=40)
print(processor.decode(output[0], skip_special_tokens=True)[len(prompt):])

In [ ]:
!pip install evaluate

In [ ]:
print("Train rows :", len(train_ds))
print("Test rows  :", len(test_ds))

Reference:

https://pyimagesearch.com/2024/12/02/fine-tune-paligemma-with-qlora-for-visual-question-answering/ \
https://blog.roboflow.com/fine-tune-paligemma-2/

In [ ]:
# -----------------------------------------------
# 1.  Configuration
# -----------------------------------------------
prompt_text  = "Describe this image."   # same prompt used at training time
batch_size   = 1                        # reduce if you hit CUDA OOM
max_tokens   = 20                       # tokens to generate per image
device       = model.device             # "cuda" for Colab GPU
strip_prefix = True                     # remove prompt from the decoded output
metric_names = ["bleu", "meteor"]       # add "cider", "spice", ... if you like

# -----------------------------------------------
# 2.  Metric loaders (evaluate library)
# -----------------------------------------------
import evaluate, numpy as np
metrics = {m: evaluate.load(m) for m in metric_names}

# -----------------------------------------------
# 3.  Helper: batched generation
# -----------------------------------------------
from tqdm.auto import tqdm
from itertools import islice
import torch

def chunk_iterable(it, size):
    "Yield successive chunks of length `size` from `it`."
    it = iter(it)
    while True:
        chunk = list(islice(it, size))
        if not chunk:
            break
        yield chunk
'''
def collate_fn(examples):
      texts = ["<image> <bos> describe this image.}" for example in examples]
      labels= [example['caption_1'] for example in examples]
      images = [example["image"].convert("RGB") for example in examples]
      tokens = processor(text=texts, images=images, suffix=labels,
      return_tensors="pt", padding="longest")
      tokens = tokens.to(torch.bfloat16).to(device)
      return tokens
'''

def generate_batch(batch):
    images   = [example["image"].convert("RGB") for example in batch]              # PIL Images
    refs     = [example['caption_1'] for example in batch]            # ground-truth
    prompts  = [prompt_text] * len(images)

    inputs = processor(images=images,
                       text=prompts,
                       return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            #do_sample=True,          # sampling → more diverse captions
           # top_p=0.95,
           # temperature=0.7,
        )

    decoded = processor.decode(outputs[0], skip_special_tokens=True)[len(prompt):]

    if strip_prefix:
        decoded = [d[len(prompt_text):].strip() if d.startswith(prompt_text) else d
                   for d in decoded]

    return decoded, refs

# -----------------------------------------------
# 4.  Iterate over the whole test split
# -----------------------------------------------
all_preds, all_refs = [], []

for batch_idxs in tqdm(chunk_iterable(range(len(test_ds)), batch_size),
                       total=(len(test_ds) + batch_size - 1) // batch_size,
                       desc="Evaluating"):
    batch = [test_ds[i] for i in batch_idxs]
    preds, refs = generate_batch(batch)
    all_preds.extend(preds)
    all_refs.extend(refs)

# -----------------------------------------------
# 5.  Compute and print metrics
# -----------------------------------------------
results = {}
for name, metric in metrics.items():
    if name == "meteor":
        # METEOR expects each ref wrapped in a list (supports multi-ref)
        score = metric.compute(predictions=all_preds,
                               references=[[r] for r in all_refs])[name]
    else:  # BLEU, CIDEr, etc.
        score = metric.compute(predictions=all_preds,
                               references=[[r] for r in all_refs])[name]
    results[name] = score

print("\n=== Evaluation on test split ===")
for k, v in results.items():
    print(f"{k.upper():<6}: {v:.4f}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Evaluating:   0%|          | 0/8905 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how 

In [ ]:
# ----- make sure lists start empty every run -----
print(type(test_ds))
eval_ds = test_ds.select(range(2000))
all_preds, all_refs = [], []

for idx, ex in enumerate(eval_ds):
    print(f"Processing sample {idx + 1}/{len(eval_ds)}")
                     # eval_ds has exactly 10 rows
    img = ex["image"]
    ref = ex["caption_1"]

    inputs = processor(img, prompt_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(**inputs,
                             max_new_tokens=20,
                             do_sample=False)      # greedy

    # out is shape [1, seq_len]; decode just that row
    pred = processor.decode(out[0], skip_special_tokens=True)

    # remove the prompt if the model echoes it
    if pred.startswith(prompt_text):
        pred = pred[len(prompt_text):].strip()

    all_preds.append(pred)
    all_refs.append(ref)

# assert len(all_preds) == len(all_refs) == 10

# ----- compute BLEU + METEOR -----
import evaluate
bleu_score = evaluate.load("bleu").compute(
    predictions=all_preds,
    references=[[r] for r in all_refs])["bleu"]

meteor_score = evaluate.load("meteor").compute(
    predictions=all_preds,
    references=[[r] for r in all_refs])["meteor"]

print(f"BLEU-4 : {bleu_score:.4f}")
print(f"METEOR : {meteor_score:.4f}")

log_file = "/content/drive/MyDrive/eval_output_pretrained_without_lora.txt"
with open(log_file, "w") as f:
    f.write("Predictions vs References:\n\n")
    for i, (pred, ref) in enumerate(zip(all_preds, all_refs)):
        f.write(f"[{i+1}] Prediction: {pred}\n")
        f.write(f"     Reference : {ref}\n\n")

    f.write(f"BLEU-4  : {bleu_score:.4f}\n")
    f.write(f"METEOR  : {meteor_score:.4f}\n")

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
Y

BLEU-4 : 0.1259
METEOR : 0.3494


In [ ]:
# -------------------------------------------
# 0.  Install/upgrade the required packages
# -------------------------------------------
# !pip install -q --upgrade datasets "transformers>=4.45" sentencepiece matplotlib

# -------------------------------------------
# 1.  Load tokenizer and dataset (streaming)
# -------------------------------------------
from datasets import load_dataset
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import itertools, os, numpy as np

BASE_MODEL = "google/paligemma-3b-mix-224"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# streaming=True  ➜ no LocalFileSystem cache error
train_stream = load_dataset(
    "caglarmert/full_riscm",
    split="train",
    streaming=True,
)

# -------------------------------------------
# 2.  Compute token lengths
# -------------------------------------------
lengths = []
prompt = "<image> <bos> describe this image. "
prompt_ids = tokenizer(prompt, add_special_tokens=False).input_ids
prompt_len = len(prompt_ids)

for example in itertools.islice(train_stream, None):  # iterate entire stream
    # strip leading/trailing whitespace
    cap = example["caption_1"].strip()
    ids  = tokenizer(cap, add_special_tokens=False).input_ids
    lengths.append(len(ids))        # *caption* length only (no prompt)

lengths = np.asarray(lengths)
print(f"Collected {len(lengths)} captions "
      f"(mean={lengths.mean():.1f}, median={np.median(lengths):.0f})")

# -------------------------------------------
# 3.  Plot histogram and save PDF
# -------------------------------------------
plt.figure(figsize=(5, 3))
bins = np.arange(0, lengths.max() + 2) - 0.5   # integer bins
plt.hist(lengths, bins=bins, edgecolor='black')
plt.xlabel("Caption length / tokens")
plt.ylabel("Frequency")
plt.title("Full-RISCM caption length distribution")
plt.tight_layout()

out_path = "length_hist.pdf"
plt.savefig(out_path)
print(f"Histogram saved ➜ {os.path.abspath(out_path)}")


In [ ]:
# -------------------- 1. install matplotlib if needed --------------------
# (comment out if already installed in the runtime)
# !pip install -q matplotlib

import re, os, matplotlib.pyplot as plt

LOG_8  = "/content/r_8_training_logs.txt"      # adjust the path if needed
LOG_16 = "/content/r=16_training_logs.txt"

def parse_log(path):
    """Return lists of (step, loss) from a plain-text log."""
    steps, losses = [], []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.lower().startswith("step"):
                continue              # skip header / empty lines
            # split on space, tab or comma
            parts = re.split(r"[,\t\s]+", line)
            if len(parts) < 2:
                continue
            try:
                step  = int(parts[0])
                loss  = float(parts[1])
                steps.append(step)
                losses.append(loss)
            except ValueError:
                continue
    return steps, losses

steps8, losses8   = parse_log(LOG_8)
steps16, losses16 = parse_log(LOG_16)

print(f"r=8  : {len(steps8)} points")
print(f"r=16 : {len(steps16)} points")

# -------------------- 2.  plot --------------------
plt.figure(figsize=(6,4))
plt.plot(steps8,  losses8,  label="LoRA r=8")
plt.plot(steps16, losses16, label="LoRA r=16")
plt.xlabel("Optimisation step")
plt.ylabel("Training loss")
plt.title("PaliGemma Q-LoRA training curves")
plt.legend()
plt.tight_layout()

out_path = "loss_curves.pdf"
plt.savefig(out_path, format="pdf")
print(f"✔  Saved figure → {os.path.abspath(out_path)}")

# also show inline
plt.show()


In [ ]:
!pip install -q wandb
!wandb login

In [ ]:
PROJECT = "DI725_ProjectPhase_3"
ENTITY  = "orcanon1-metu"

In [ ]:
import json, re, wandb

def replay_eval_metrics(path, run_name, group=None):
    """
    Accepts two formats:
      1) a single-line JSON dict  {"bleu":0.102,"meteor":0.313}
      2) a plain line such as    BLEU: 0.1020, METEOR: 0.3133
    Logs the metrics to a fresh W&B run.
    """
    bleu = meteor = None

    with open(path) as f:
        for line in f:
            s = line.strip()
            if not s:
                continue
            # --- try strict JSON first
            if s.startswith("{") and s.endswith("}"):
                try:
                    metrics = json.loads(s)
                    bleu   = metrics.get("bleu")   or metrics.get("BLEU")
                    meteor = metrics.get("meteor") or metrics.get("METEOR")
                    break
                except json.JSONDecodeError:
                    pass
            # --- fall back to regex extraction
            m_bleu   = re.search(r"bleu[:=\s]+([\d.]+)",   s, re.I)
            m_meteor = re.search(r"meteor[:=\s]+([\d.]+)", s, re.I)
            if m_bleu and m_meteor:
                bleu   = float(m_bleu.group(1))
                meteor = float(m_meteor.group(1))
                break

    if bleu is None or meteor is None:
        raise ValueError(f"Could not parse metrics in {path}")

    run = wandb.init(project=PROJECT, entity=ENTITY,
                     name=run_name, group=group,
                     job_type="offline-eval")
    wandb.log({"bleu": bleu, "meteor": meteor}, step=0)
    run.save(path)
    run.finish()


In [ ]:
import re, wandb

PROJECT = "DI725_ProjectPhase_3"
ENTITY  = "orcanon1-metu"          # adjust if needed

def push_bleu_meteor(txt_path, run_name, group=None):
    """
    Parse the last two lines:
        BLEU-4  : 0.1020
        METEOR  : 0.3133
    and log them to a new W&B run.
    """
    bleu = meteor = None
    with open(txt_path) as f:
        for line in f:
            m_bleu   = re.search(r"BLEU[-\s]*4\s*[:=]\s*([\d.]+)", line, re.I)
            m_meteor = re.search(r"METEOR\s*[:=]\s*([\d.]+)", line, re.I)
            if m_bleu:
                bleu = float(m_bleu.group(1))
            if m_meteor:
                meteor = float(m_meteor.group(1))

    if bleu is None or meteor is None:
        raise ValueError(f"Could not find BLEU or METEOR in {txt_path}")

    run = wandb.init(project=PROJECT, entity=ENTITY,
                     name=run_name, group=group,
                     job_type="offline-eval")
    wandb.log({"bleu": bleu, "meteor": meteor}, step=0)
    run.save(txt_path)        # attach raw log
    run.finish()
    print(f"Logged {run_name}: BLEU={bleu}, METEOR={meteor}")

# ---- push the three eval logs ----
push_bleu_meteor("/content/eval_output_pretrained_without_lora.txt",
                 "eval_PG_base",   group="Evaluation")
push_bleu_meteor("/content/eval_output_r=8.txt",
                 "eval_QLoRA_r8",  group="Evaluation")
push_bleu_meteor("/content/eval_output_r16.txt",
                 "eval_QLoRA_r16", group="Evaluation")
